# Train a Model from Scratch and Deploy to Device

This notebook will walk you through all the steps required to train up a brand new model and deploy it to your device.
As an amusing motivating example, we will do the following:

1. Download COCO dataset.
1. Use Deep Dream to convert it into a funny-looking dataset.
1. Train a Pix2Pix GAN generator to convert COCO images into dreamified images.
1. Convert the generator model into OpenVINO.
1. Modify the device's azureeyemodule to accept this model.
1. Download this model to the Percept DK.

## Download the GitHub repo

In [ ]:
topdir = !pwd
topdir = topdir[0]
topdir

In [ ]:
!git clone https://github.com/microsoft/azure-percept-advanced-development.git
%cd azure-percept-advanced-development
!git checkout strangem-tutorial
!git pull
%cd tutorials/pytorch-from-scratch-tutorial

## Make the Dataset

To make the dataset, we are going to download the COCO validation split (because we don't need the full COCO dataset for this),
then run Deep Dream on it. Finally, there's some further processing we need to do to get it into a format that Pix2Pix will accept for training.

In [ ]:
!pip install wget
!python make_dataset.py --nimgs 1000 --nvalimgs 250 --destination $topdir/coco-dreamified

# This script will take several hours to complete. If it stops working because AML disconnects,
# you can call it again, but this time use the following line instead:
# !python make_dataset.py --nimgs 1000 --nvalimgs 250 --destination $topdir/coco-dreamified --coco $topdir/coco-dreamified --continue

In [ ]:
# We have to make sure that image A is the same size as image B
# I.e., we need to make sure that the dreamified image A is the same size as its corresponding non-dreamified image B.
!python resize.py $topdir/coco-dreamified

In [ ]:
# The Pix2Pix model tutorials all require that you put the dreamified and the raw images into the same image,
# along the x axis. We'll do that here, using the Pix2Pix repository's script. See it's docstring for license information.
!python ./combine_A_and_B.py --fold_A $topdir/coco-dreamified/A --fold_B $topdir/coco-dreamified/B --fold_AB $topdir/coco-dreamified-combined --no_multiprocessing

In [ ]:
# Visualize the results
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 20)
from PIL import Image
import os
import random

# Pick an image at random
datadpath = os.path.join(topdir, "coco-dreamified-combined", "train")
fpaths = [os.path.join(datadpath, fname) for fname in os.listdir(datadpath)]
random.shuffle(fpaths)

# Display it
img = Image.open(fpaths[0])
plt.imshow(img)
plt.show()

## Train the GAN to Generate Dreamified Images from Real Ones

You should now have a dataset in a folder called "coco-dreamifed-combined". Let's make an AML experiment that trains Pix2Pix
to image translate between the two domains.

In [ ]:
# Use the default datasore associated with the current workspace
from azureml.core import Workspace
from azureml.core import Dataset

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

In [ ]:
# Upload the dataset to the datastore
import os
datastore_data_path = "datasets/coco-dreamified-combined"
datastore.upload(src_dir=os.path.join(topdir, "coco-dreamified-combined"), target_path=datastore_data_path)

In [ ]:
# Register the uploaded data as an AML dataset
dataset = Dataset.File.from_files(path=(datastore, datastore_data_path))
dataset = dataset.register(workspace=ws, name="coco-dreamified-combined", description="COCO after dreamification")

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config() 

# Choose a name for your compute cluster
cluster_name = "gpu1"

# Verify that the cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',  # Make sure to choose something your subscription has access to
                                                           idle_seconds_before_scaledown=2400,
                                                           min_nodes=0,
                                                           max_nodes=1)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

In [ ]:
# This is where we are going to put everything
root_outputs_path = os.path.join(topdir, "outputs")

model_path = os.path.join(root_outputs_path, "model")
os.makedirs(model_path, exist_ok=True)

ir_output_path = os.path.join(root_outputs_path, "intel")
os.makedirs(ir_output_path, exist_ok=True)

In [ ]:
# Download the training script from an external repository (under a BSD-like license)
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix
!git checkout f13aab8148bd5f15b9eb47b690496df8dadbab0c

In [ ]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset

ws = Workspace.from_config()
dataset = Dataset.get_by_name(workspace=ws, name="coco-dreamified-combined")
experiment = Experiment(workspace=ws, name="dreamification-experiment")

# We are going to run the train.py script with some args. See the train.py script for license information.
config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=cluster_name,
    arguments=[
        "--dataroot", dataset.as_named_input("input").as_mount(),
        "--name", "dreamifier",
        "--n_epochs", 50,
        "--n_epochs_decay", 25,
        "--checkpoints_dir", "outputs/checkpoints",
        "--model", "pix2pix",  # Use Pix2Pix instead of CycleGAN
        "--direction", "AtoB"  # Train the network to convert raw images to dreamified ones
    ]
)

# Set up the training environment.
env = Environment.from_pip_requirements(name="PyTorch-Pix2Pix-Env", file_path="requirements.txt")
config.run_config.environment = env

run = experiment.submit(config)
aml_url = run.get_portal_url()
print("Submitted to compute cluster. Click link below.")
print(aml_url)

In [ ]:
# Wait until the experiment run is completed.
run.wait_for_completion(show_output=True)

In [ ]:
# Get the model from the latest completed run outputs
import glob
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name='dreamification-experiment')
runs = experiment.get_runs()

completed_run = None
for r in runs:
    if r.get_status() == 'Completed':
        completed_run = r
        break

if completed_run is None:
    print("No completed run available")
else:
    !rm -rf model_path
    completed_run.download_files("outputs", model_path)
    print(f'Downloaded model file: {model_path}')

## Convert to ONNX

In order to convert to ONNX, we'll apply a small patch to one of the files in the repository,
so that we can get an ONNX model out when we test it. See https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/issues/1113

In [ ]:
%%writefile 
$topdir/azure-percept-advanced-development/tutorials/pytorch-from-scratch-tutorial/pytorch-CycleGAN-and-pix2pix/0001-Update-base-model.py.patchFrom de494caeb99dcb457c559222277f87b549b90436 Mon Sep 17 00:00:00 2001
From: Max Strange
Date: Fri, 19 Mar 2021 14:36:09 -0700
Subject: [PATCH] Temp Commit

---
 models/base_model.py | 12 ++++++++++++
 1 file changed, 12 insertions(+)

diff --git a/models/base_model.py b/models/base_model.py
index 6de961b..304bd3d 100644
--- a/models/base_model.py
+++ b/models/base_model.py
@@ -198,6 +198,18 @@ class BaseModel(ABC):
                     self.__patch_instance_norm_state_dict(state_dict, net, key.split('.'))
                 net.load_state_dict(state_dict)
 
+                # Create ONNX model
+                net.eval()
+                batch_size = 1
+                input_shape = (3, 256, 256)  # Default crop size
+                export_onnx_file = load_filename[:-4]+".onnx"
+                save_path = os.path.join(self.save_dir, export_onnx_file)
+
+                dinput = torch.randn(batch_size, *input_shape)
+                torch.onnx.export(net, dinput, save_path)
+
+                print('The ONNX file ' + export_onnx_file + ' is saved at %s' % save_path)
+
     def print_networks(self, verbose):
         """Print the total number of parameters in the network and (if verbose) network architecture
 
-- 
2.30.1



In [ ]:
# Now apply the patch
!git apply $topdir/azure-percept-advanced-development/tutorials/pytorch-from-scratch-tutorial/pytorch-CycleGAN-and-pix2pix/0001-Update-base-model.py.patch

In [ ]:
!cat $topdir/azure-percept-advanced-development/tutorials/pytorch-from-scratch-tutorial/pytorch-CycleGAN-and-pix2pix/models/base_model.py

## Test with our Val Split

The generator should be trained. Let's load it in and test it on some example images.

In [ ]:
dataroot = os.path.join(topdir, "coco-dreamified-combined")
results_dir = os.path.join(topdir, "outputs", "results")

# Pix2Pix framework wants our val split to be called test split. I've heard it both ways :)
!mv $dataroot/val $dataroot/test
!pip install dominate
!python test.py --dataroot $dataroot --direction AtoB --model pix2pix --name dreamifier --checkpoints_dir $topdir/outputs/model/outputs/checkpoints --results_dir $results_dir --gpu_ids -1

In [ ]:
# Visualize the results
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 20)
from PIL import Image
import os
import random

# Pick an image at random
images_dir = os.path.join(results_dir, "dreamifier", "test_latest", "images")
fpaths = [os.path.join(images_dir, fname) for fname in os.listdir(images_dir) if fname.endswith("_fake_B.png")]
random.shuffle(fpaths)

# Display a few
nimgs = 5
for i in range(0, nimgs * 2, 2):
    # Input image on the left
    imgname = os.path.basename(fpaths[i])
    rawfpath = os.path.join(images_dir, imgname.replace("_fake_B.png", "_real_A.png"))
    raw = Image.open(rawfpath)
    plt.subplot(nimgs, 2, i + 1)
    plt.imshow(raw)

    # Dreamified image on the right
    img = Image.open(fpaths[i])
    plt.subplot(nimgs, 2, i + 2)
    plt.imshow(img)
plt.show()

## Check that the ONNX Model Works Too

In [ ]:
%pip install onnxruntime

import os
import onnx
onnx_model_path = os.path.join(topdir, "outputs", "model", "outputs", "checkpoints", "dreamifier", "latest_net_G.onnx")
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

In [ ]:
# Validate with the ONNX model now
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 20)
from PIL import Image
import numpy as np
import os
import onnxruntime
import torch

# Make predictions with our ONNX model
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
ort_session = onnxruntime.InferenceSession(onnx_model_path)

nimgs = 5
for i in range(0, nimgs * 2, 2):
    # Input image on the left
    imgname = os.path.basename(fpaths[i])
    rawfpath = os.path.join(images_dir, imgname.replace("_fake_B.png", "_real_A.png"))
    raw = Image.open(rawfpath)
    plt.subplot(nimgs, 2, i + 1)
    plt.imshow(raw)

    # Run the raw image through the model and display on the right
    # We need to do some pre and post processing to make the network happy
    # First, convert the raw image from Numpy to Torch, BCWH format, and then convert to float.
    raw_as_tensor = torch.from_numpy(np.asarray(raw)).permute(2, 0, 1).unsqueeze(0).type(torch.FloatTensor)
    # Now go from [0, 255] -> [-1.0, 1.0]
    raw_as_tensor = (raw_as_tensor - 127.5) / (127.5)
    # Create an ONNX RT session and feed it the tensor image
    ort_output = ort_session.run(None, {ort_session.get_inputs()[0].name: to_numpy(raw_as_tensor)})
    # Remove the batch dimension, permute it back to WHC format, and then convert to int, range [0, 255]
    # The output of this model is [-1.0, 1.0]
    img = ort_output[0].squeeze(0)
    img = torch.from_numpy(img).permute(1, 2, 0).numpy()
    img = np.round((img + 1) * 255 / 2)
    print("Max:", np.max(img), "Min:", np.min(img))
    img = img.astype(np.uint8)
    plt.subplot(nimgs, 2, i + 2)
    plt.imshow(img)

plt.show()

In [ ]:
    # We need this for converting to OpenVINO
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name

## Convert to OpenVINO

In [ ]:
# First convert from ONNX to IR
!rm -rf $topdir/openvino/mnt
!mkdir -p $topdir/openvino/mnt
!cp $onnx_model_path $topdir/openvino/mnt/model.onnx
!docker run --rm -v $topdir/openvino/mnt:/mnt -w /mnt openvino/ubuntu18_dev:2021.1 \
    python3 "/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py" \
    --input_model "./model.onnx" -o "./" --input $input_name --output $output_name --scale 127.5 --mean_values "(127.5, 127.5, 127.5)"
!cp $topdir/openvino/mnt/model.bin $ir_output_path
!cp $topdir/openvino/mnt/model.xml $ir_output_path

In [ ]:
!ls $ir_output_path